In [65]:
import json
from typing import List
from unstructured.chunking.title import chunk_by_title
from unstructured.partition.pdf import partition_pdf

In [2]:
from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings

In [101]:
load_dotenv()

True

In [3]:
def partition_docuents(file_path:str):


    elements=partition_pdf(
        filename=file_path,
        strategy="hi_res",
        infer_table_structure=True,
        extract_image_block_types=["Images"],
        extract_image_block_to_payload=True
    )
    print(f"Partition of the {file_path} is completed")
    return elements
    

In [4]:
elements=partition_docuents("attention-is-all-you-need.pdf")

Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 3271.69it/s]
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
The requested type (Images) doesn't match any available type


Partition of the attention-is-all-you-need.pdf is completed


In [7]:
len(elements)

220

In [13]:
elements[32].to_dict()

{'type': 'NarrativeText',
 'element_id': '3ee6b319d76aec309ece8b130d9ce347',
 'text': '∗Equal contribution. Listing order is random. Jakob proposed replacing RNNs with self-attention and started the effort to evaluate this idea. Ashish, with Illia, designed and implemented the first Transformer models and has been crucially involved in every aspect of this work. Noam proposed scaled dot-product attention, multi-head attention and the parameter-free position representation and became the other person involved in nearly every detail. Niki designed, implemented, tuned and evaluated countless model variants in our original codebase and tensor2tensor. Llion also experimented with novel model variants, was responsible for our initial codebase, and efficient inference and visualizations. Lukasz and Aidan spent countless long days designing various parts of and implementing tensor2tensor, replacing our earlier codebase, greatly improving results and massively accelerating our research.',
 'met

In [21]:
set([str(type(el)) for el in elements])


{"<class 'unstructured.documents.elements.FigureCaption'>",
 "<class 'unstructured.documents.elements.Footer'>",
 "<class 'unstructured.documents.elements.Formula'>",
 "<class 'unstructured.documents.elements.Header'>",
 "<class 'unstructured.documents.elements.Image'>",
 "<class 'unstructured.documents.elements.ListItem'>",
 "<class 'unstructured.documents.elements.NarrativeText'>",
 "<class 'unstructured.documents.elements.Table'>",
 "<class 'unstructured.documents.elements.Text'>",
 "<class 'unstructured.documents.elements.Title'>"}

In [ ]:
def create_chunks_by_title(elements):
    """Create intelligent chunks using title-based strategy"""
    print("🔨 Creating smart chunks...")
    
    chunks = chunk_by_title(
        elements, # The parsed PDF elements from previous step
        new_after_n_chars=2400, # Try to start a new chunk after 2400 characters
        combine_text_under_n_chars=500 # Merge tiny chunks under 500 chars with neighbors
    )
    
    print(f"✅ Created {len(chunks)} chunks")
    return chunks

# Create chunks
chunks = create_chunks_by_title(elements)


🔨 Creating smart chunks...
✅ Created 110 chunks


In [87]:
chunks[11].metadata.orig_elements[-1]

In [85]:
type(chunks[11]).__name__

'CompositeElement'

In [95]:
def separate_content_types(chunk):

    content_data={
        'text':chunk.text,
        'images':[],
        'tables':[],
        'types':['text']
    }

    if hasattr(chunk,'metadata') and hasattr(chunk.metadata,'orig_elements'):
        for element in chunk.metadata.orig_elements:
            element_type=type(element).__name__
            if element_type=="Table":
                content_data['types'].append('table')
                table_html=getattr(element.metadata,'text_as_html',element.text)
                content_data['tables'].append(table_html)
            elif element_type=="Image":
                if hasattr(element,'metadata') and hasattr(element.metadata,'image_base64'):
                    content_data["types"].append('image')
                    content_data["types"].append(element.metadata.image_base64)
    content_data['types']=list(set(content_data['types']))

    return content_data


def create_ai_summary(text:str,tables:list[str],images:list[str]):
    """crete a  ai summary """
    try:

        llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
        prompt_text = f"""You are creating a searchable description for document content retrieval.

        CONTENT TO ANALYZE:
        TEXT CONTENT:
        {text}

        """

        if tables:
            prompt_text += "TABLES:\n"
            for i,table in tables:
                prompt_text+=table
                prompt_text += """
                YOUR TASK:
                Generate a comprehensive, searchable description that covers:

                1. Key facts, numbers, and data points from text and tables
                2. Main topics and concepts discussed  
                3. Questions this content could answer
                4. Visual content analysis (charts, diagrams, patterns in images)
                5. Alternative search terms users might use

                Make it detailed and searchable - prioritize findability over brevity.

                SEARCHABLE DESCRIPTION:"""
                message_content = [{"type": "text", "text": prompt_text}]

                for image_base64 in images:
                    message_content.append({
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"}
                    })
                message = HumanMessage(content=message_content)
                response = llm.invoke([message])
        
        return response.content
        
    except Exception as e:
        print(f"     ❌ AI summary failed: {e}")
        # Fallback to simple summary
        summary = f"{text[:300]}..."
        if tables:
            summary += f" [Contains {len(tables)} table(s)]"
        if images:
            summary += f" [Contains {len(images)} image(s)]"
        return summary
            

In [96]:
def summarise_chunks(chunks):
    """Process all chunks with AI Summaries"""
    print("🧠 Processing chunks with AI Summaries...")
    
    langchain_documents = []
    total_chunks = len(chunks)
    
    for i, chunk in enumerate(chunks):
        current_chunk = i + 1
        print(f"   Processing chunk {current_chunk}/{total_chunks}")
        
        # Analyze chunk content
        content_data = separate_content_types(chunk)
        
        # Debug prints
        print(f"     Types found: {content_data['types']}")
        print(f"     Tables: {len(content_data['tables'])}, Images: {len(content_data['images'])}")
        
        # Create AI-enhanced summary if chunk has tables/images
        if content_data['tables'] or content_data['images']:
            print(f"     → Creating AI summary for mixed content...")
            try:
                enhanced_content = create_ai_summary(
                    content_data['text'],
                    content_data['tables'], 
                    content_data['images']
                )
                print(f"     → AI summary created successfully")
                print(f"     → Enhanced content preview: {enhanced_content[:200]}...")
            except Exception as e:
                print(f"     ❌ AI summary failed: {e}")
                enhanced_content = content_data['text']
        else:
            print(f"     → Using raw text (no tables/images)")
            enhanced_content = content_data['text']
        
        # Create LangChain Document with rich metadata
        doc = Document(
            page_content=enhanced_content,
            metadata={
                "original_content": json.dumps({
                    "raw_text": content_data['text'],
                    "tables_html": content_data['tables'],
                    "images_base64": content_data['images']
                })
            }
        )
        
        langchain_documents.append(doc)
    
    print(f"✅ Processed {len(langchain_documents)} chunks")
    return langchain_documents


# Process chunks with AI
processed_chunks = summarise_chunks(chunks)

🧠 Processing chunks with AI Summaries...
   Processing chunk 1/110
     Types found: ['text']
     Tables: 0, Images: 0
     → Using raw text (no tables/images)
   Processing chunk 2/110
     Types found: ['text']
     Tables: 0, Images: 0
     → Using raw text (no tables/images)
   Processing chunk 3/110
     Types found: ['text']
     Tables: 0, Images: 0
     → Using raw text (no tables/images)
   Processing chunk 4/110
     Types found: ['text']
     Tables: 0, Images: 0
     → Using raw text (no tables/images)
   Processing chunk 5/110
     Types found: ['text']
     Tables: 0, Images: 0
     → Using raw text (no tables/images)
   Processing chunk 6/110
     Types found: ['text']
     Tables: 0, Images: 0
     → Using raw text (no tables/images)
   Processing chunk 7/110
     Types found: ['text']
     Tables: 0, Images: 0
     → Using raw text (no tables/images)
   Processing chunk 8/110
     Types found: ['text']
     Tables: 0, Images: 0
     → Using raw text (no tables/images)

In [104]:
def create_vector_store(documents, persist_directory="dbv1/chroma_db"):
    """Create and persist ChromaDB vector store"""
    print("🔮 Creating embeddings and storing in ChromaDB...")
        
    embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    
    # Create ChromaDB vector store
    print("--- Creating vector store ---")
    vectorstore = Chroma.from_documents(
        documents=documents,
        embedding=embedding_model,
        persist_directory=persist_directory, 
        collection_metadata={"hnsw:space": "cosine"}
    )
    print("--- Finished creating vector store ---")
    
    print(f"✅ Vector store created and saved to {persist_directory}")
    return vectorstore

# Create the vector store
db = create_vector_store(processed_chunks)

🔮 Creating embeddings and storing in ChromaDB...
--- Creating vector store ---
--- Finished creating vector store ---
✅ Vector store created and saved to dbv1/chroma_db


In [106]:
query = "What are the two main components of the Transformer architecture? "
retriever = db.as_retriever(search_kwargs={"k": 3})
chunks = retriever.invoke(query)

# Export to JSON
chunks

[Document(id='90bedde1-af9b-4a3e-98cf-c7d54dcd12ca', metadata={'original_content': '{"raw_text": "The Transformer follows this overall architecture using stacked self-attention and point-wise, fully connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1, respectively.\\n\\n3.1 Encoder and Decoder Stacks", "tables_html": [], "images_base64": []}'}, page_content='The Transformer follows this overall architecture using stacked self-attention and point-wise, fully connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1, respectively.\n\n3.1 Encoder and Decoder Stacks'),
 Document(id='49051ee7-303d-4232-a1a4-d84b07de9b8e', metadata={'original_content': '{"raw_text": "Transformer generalizes well to other tasks by applying it successfully to English constituency parsing both with large and limited training data.", "tables_html": [], "images_base64": []}'}, page_content='Transformer generalizes well to other 